In [62]:
import pandas as pd
import os

In [63]:
names = ['user_id', 'movie_id', 'rating', 'timestamp']
train1 = pd.read_csv(os.path.join('ml-100k', 'u5.base'), '\t', names=names,engine='python')
train1.drop(['timestamp'], axis = 1)

,user_id,movie_id,rating
0,1,1,5
1,1,2,3
2,1,4,3
3,1,5,3
4,1,6,5
...,...,...,...
79995,943,943,5
79996,943,1011,2
79997,943,1067,2
79998,943,1074,4


In [64]:
test1 = pd.read_csv(os.path.join('ml-100k', 'u5.test'), '\t', names=names,engine='python')
test1.drop(['timestamp'], axis = 1)

,user_id,movie_id,rating
0,1,3,4
1,1,13,5
2,1,15,5
3,1,18,4
4,1,19,5
...,...,...,...
19995,943,1028,2
19996,943,1044,3
19997,943,1047,2
19998,943,1228,3


In [65]:
comp_data = pd.read_csv(os.path.join('ml-100k', 'u5.base'), '\t', names=names,engine='python')
user_data = comp_data.pivot(index = 'user_id', columns = 'movie_id', values = 'rating').fillna(0)
user_data.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1662,1663,1664,1666,1669,1672,1673,1677,1678,1679
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,0.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# User Similarity Matrix using Cosine similarity as a similarity measure between Users
user_similarity = cosine_similarity(user_data)
user_similarity[np.isnan(user_similarity)] = 0
print(user_similarity)
print(user_similarity.shape)

[[1.         0.15068839 0.05234295 ... 0.1588403  0.09823454 0.22729015]
 [0.15068839 1.         0.10553422 ... 0.03505581 0.11992342 0.09181167]
 [0.05234295 0.10553422 1.         ... 0.11195523 0.08402349 0.02913911]
 ...
 [0.1588403  0.03505581 0.11195523 ... 1.         0.07031712 0.18022618]
 [0.09823454 0.11992342 0.08402349 ... 0.07031712 1.         0.07526602]
 [0.22729015 0.09181167 0.02913911 ... 0.18022618 0.07526602 1.        ]]
(943, 943)


In [67]:
neighbours = user_similarity[user_similarity[:,1] > 0.5]

In [68]:
user_similarity[user_similarity< 0.5] = 0.0
#ini_array1[ini_array1<0] = 0

In [69]:
print(user_similarity)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [70]:
print(user_similarity.shape)


(943, 943)


In [71]:
train1_piv = train1.pivot(index = 'user_id', columns = 'movie_id', values = 'rating').fillna(0)
test1_piv = test1.pivot(index = 'user_id', columns = 'movie_id', values = 'rating').fillna(0)

In [72]:
user_predicted_ratings = np.dot(user_similarity, user_data)
user_predicted_ratings

array([[11.12401894,  7.58693338,  2.02982688, ...,  0.        ,
         0.        ,  0.        ],
       [ 4.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [73]:
user_predicted_ratings.shape

(943, 1650)

In [74]:
dummy_train = train1.copy()
dummy_test = test1.copy()

dummy_train['rating'] = dummy_train['rating'].apply(lambda x: 0 if x > 0 else 1)
dummy_test['rating'] = dummy_test['rating'].apply(lambda x: 1 if x > 0 else 0)

In [75]:
user_predicted_ratings.shape

(943, 1650)

In [76]:
# The movies not rated by user is marked as 1 for prediction 
dummy_train = dummy_train.pivot(index = 'user_id', columns = 'movie_id', values = 'rating').fillna(1)

# The movies not rated by user is marked as 0 for evaluation 
dummy_test = dummy_test.pivot(index ='user_id', columns = 'movie_id', values = 'rating').fillna(0)

In [77]:
user_final_ratings = np.multiply(user_predicted_ratings, dummy_train)
user_final_ratings.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1662,1663,1664,1666,1669,1672,1673,1677,1678,1679
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,2.029827,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [78]:
test_user_features = test1.pivot(index = 'user_id', columns = 'movie_id', values = 'rating').fillna(0)
test_user_similarity = cosine_similarity(test_user_features)
test_user_similarity[np.isnan(test_user_similarity)] = 0

print(test_user_similarity)
print("- "*10)
print(test_user_similarity.shape)

[[1.         0.         0.         ... 0.07061981 0.         0.07750815]
 [0.         1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.07061981 0.         0.         ... 1.         0.04318959 0.05270306]
 [0.         0.         0.         ... 0.04318959 1.         0.09734412]
 [0.07750815 0.         0.         ... 0.05270306 0.09734412 1.        ]]
- - - - - - - - - - 
(927, 927)


In [79]:
test_user_similarity[test_user_similarity< 0.5] = 0.0

In [80]:
user_predicted_ratings_test = np.dot(test_user_similarity, test_user_features)
user_predicted_ratings_test

array([[0., 0., 4., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [81]:
test_user_final_rating = np.multiply(user_predicted_ratings_test, dummy_test)
test_user_final_rating.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1667,1668,1670,1671,1674,1675,1676,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [82]:
train1['rating'].describe()

count    80000.000000
mean         3.531538
std          1.127418
min          1.000000
25%          3.000000
50%          4.000000
75%          4.000000
max          5.000000
Name: rating, dtype: float64

In [83]:
from sklearn.preprocessing import MinMaxScaler

X = test_user_final_rating.copy() 
X = X[X > 0] # only consider non-zero values as 0 means the user haven't rated the movies

scaler = MinMaxScaler(feature_range = (0.5, 5))
scaler.fit(X)
pred = scaler.transform(X)

print(pred)

[[      nan       nan 3.875     ...       nan       nan       nan]
 [      nan       nan       nan ...       nan       nan       nan]
 [      nan       nan       nan ...       nan       nan       nan]
 ...
 [2.2440657       nan       nan ...       nan       nan       nan]
 [      nan       nan       nan ...       nan       nan       nan]
 [      nan 5.              nan ...       nan       nan       nan]]


In [84]:
total_non_nan = np.count_nonzero(~np.isnan(pred))
total_non_nan

20000

In [85]:
test = test1.pivot(index = 'user_id', columns = 'movie_id', values = 'rating')
test.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1667,1668,1670,1671,1674,1675,1676,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
# Mean abslute error

mae = np.abs(pred - test).sum().sum()/total_non_nan
print(mae)

0.7397438027564948
